# Notes
WSZYSTKIE PLIKI SĄ IDENTYCZNE. WSZYSTKIE TO EXCEL - 14 kart
Czyszczenie:
•	Wszystkie NaN na 0
•	Skalowanie przed modelem
•	Inżynieria cech (wskaźniki ekonomiczne, różnica miedzy każym z elementów)

TESTY DO PRZEPROWADZENIA:
•	Uwzglednienie wszystkiego
•	Skupienie sie na zmianach kapitałowych
•	Ograniczenie tylko główne wartości
•	Grupowanie po przedsiębiorstwie? 

Podsumowanie co wyciągnąć z pliku:
Dodatkowa kolumna jako nazwa pliku
C – nazwa kolumny w DF


In [266]:
import pandas as pd
import re

In [267]:
names_data = pd.read_csv('dataframes/names_gpw.csv')

In [268]:
def clean_column_names(columns):
    cleaned_columns = []
    for column in columns:
        column = column.lower()
        column = re.sub(r'\(.*?\)', '', column)
        column = re.sub(r'[^\w\s]', '', column)
        column = re.sub(r'\s+', '_', column)
        cleaned_columns.append(column)
    return cleaned_columns

In [269]:
def count_rows_by_filename(df):
    counts = df.groupby('filename').size().reset_index(name='count')
    return counts

# FULL DF CLEARNING

In [270]:
full_data = pd.read_csv("dataframes/dataframe_gpw.csv")

In [271]:
full_merged_df = pd.merge(full_data, names_data, on='filename')

In [272]:
full_merged_df = full_merged_df.sort_values(by=['filename', 'End of period'])

In [273]:
full_merged_df.columns = clean_column_names(full_merged_df.columns)

In [274]:
count_rows_by_filename(full_merged_df)

,filename,count
0,11BIT.xlsx,49
1,3LPSA.xlsx,6
2,3RGAMES.xlsx,48
3,AB.xlsx,70
4,AC.xlsx,51
...,...,...
332,ZPUE.xlsx,95
333,ZREMBCH.xlsx,65
334,ZUE.xlsx,53
335,ZUK.xlsx,67


In [275]:
# Delete all companies under 4 because its not enough to cover any change in assets or liabilities
filenames_to_remove = [
    'SADOVAYA.xlsx',
    'BMW.xlsx',
    'KERNEL.xlsx',
    'SILVANO.xlsx',
    'IMCSA.xlsx',
    'STSHOLDING.xlsx'
]

df_filtered = full_merged_df[~full_merged_df['filename'].isin(filenames_to_remove)]

## Missing data

In [276]:
full_merged_df.isnull().sum()

end_of_period                                                      0
revenues_from_sales                                              945
operating_profitloss                                             737
net_profitloss_attributable_to_equity_holders_of_the_parent      577
total_comprehensive_income_attributable_to_equity_holders      17723
                                                               ...  
current_financial_liabilities                                  19906
noncurrent_financial_liabilities                               19906
financial_liabilities_                                         19906
company_name                                                       0
sector                                                            13
Length: 108, dtype: int64

In [277]:
# Sektor NaN
rows_with_nan_in_sektor = full_merged_df[full_merged_df['sector'].isna()]
rows_with_nan_in_sektor

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,aktywa,...,debt_service_ratio,rate_debt_security,leverage_,asset_utilization_,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,financial_liabilities_,company_name,sector
49,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.301800e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
50,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.053760e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
51,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.613490e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
52,2020-12-31,34515.00000,3929.00000,138.00000,138.0,4114.00000,8513.00000,-3261.00000,-5264.0000,1.640630e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
53,2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.547040e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
54,2021-12-31,39704.00000,3898.00000,2169.00000,2169.0,4459.00000,5831.00000,-2718.00000,-3195.0000,1.571200e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
2522,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166580e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN
2523,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.295270e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN
18362,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.065330e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN
18363,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.348397e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN


# Notes
Manually checking for each company

3LP SA - handel
BMW AG - motoryzacja
Uf Games SA - gry


In [278]:
company_to_sector = {
    '3LP SA': 'handel',
    'BMW AG': 'motoryzacja',
    'Uf Games SA': 'gry'
}

full_merged_df['sector'] = full_merged_df.apply(lambda row: company_to_sector[row['company_name']] if pd.isna(row['sector']) else row['sector'], axis=1)

In [279]:
# Rest of financial data - fill NaN with 0 (= no change)
full_merged_df = full_merged_df.fillna(0)

In [280]:
full_merged_df

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,aktywa,...,debt_service_ratio,rate_debt_security,leverage_,asset_utilization_,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,financial_liabilities_,company_name,sector
0,2010-03-31,66.0,0.0,-16.0,0.0,29.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry
1,2010-09-30,129.0,0.0,-26.0,0.0,6.0,0.0,0.0,0.0,1411.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry
2,2010-12-31,228.0,0.0,-198.0,0.0,0.0,0.0,0.0,0.0,1352.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry
3,2011-03-31,144.0,0.0,-50.0,0.0,0.0,0.0,0.0,0.0,1342.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry
4,2011-06-30,959.0,0.0,377.0,0.0,7.0,0.0,0.0,0.0,1659.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19901,2021-09-30,995385.0,103668.0,104744.0,106250.0,36985.0,354390.0,-22037.0,-433167.0,2649304.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje
19902,2021-12-31,733974.0,46890.0,32175.0,42434.0,42311.0,50929.0,-65863.0,30091.0,2508447.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje
19903,2022-03-31,696778.0,6649.0,-1246.0,41367.0,38391.0,13967.0,-33280.0,57099.0,2741947.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje
19904,2022-06-30,1209418.0,101579.0,92625.0,45386.0,39932.0,101877.0,-21179.0,-81990.0,2981239.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje


In [281]:
# Change each indicator to % value


## Feature Engineering

In [282]:
# Change in capital or assets structure
# feature: change in last quarter for each company (every major data attribute)
# feature: change in last two quarters for each company (every major data attribute)
# feature: change in last four quarters for each company (every major data attribute)
# feature: sum of liability 
# feature: sum of assets

# NN DATA CLEANING

In [283]:
nn_data = pd.read_csv("dataframes/nn_gpw_data.csv")

In [284]:
nn_merged_df = pd.merge(nn_data, names_data, on='filename')

In [285]:
nn_merged_df = nn_merged_df.sort_values(by=['filename', 'End of period'])

In [286]:
nn_merged_df.columns = clean_column_names(nn_merged_df.columns)